# Multi-step pyramidal imaging

In this notebook, we implement a pyramidal, Google Maps-like, imaging scheme: We start with a coarse image with large pixel size, divide it into tiles and then only image tiles that contain at least some structure at higher resolution (smaller pixel size, though increasing STED power could also be implemented).

First, we define a function that investigates an image and returns bounding boxes of sub-images that should be imaged at higher resolution:

In [ ]:
import numpy as np
import specpy
from itertools import pairwise, product


def divide_into_tiles(img, num_divisions=2, threshold=0, min_size_fraction=0.05):
    """
    Divide an image into tiles and check if a minimum of tile area is above a set threshold.
    Return bounding boxes for each tile that matches this criterion.
    """

    # num_divisions can be sequence (per dimension)
    # if only a scalar is given, we re-use it for all dimensions
    if np.isscalar(num_divisions):
        num_divisions = [num_divisions] * img.ndim

    # get all candidate bboxes
    bboxes = []
    start_end_per_dimension = [
        map(list, pairwise(np.linspace(0, s, n + 1)))
        for s, n in zip(img.shape, num_divisions)
    ]
    for bbox in map(list, product(*start_end_per_dimension)):
        bboxes.append(np.array(bbox).T)

    # select only the ones with enough intensity in img
    bboxes_above_thresh = []
    for bbox in bboxes:
        # select tile defined by bbox rounded to nearest pixel
        start, end = bbox
        start = np.round(start).astype(int)
        end = np.round(end).astype(int)
        tile = img[tuple(slice(s, e) for s, e in zip(start, end))]
        # if fraction of pixels above threshold is above minimal size fraction,
        # accept into final list of bboxes
        if (tile >= threshold).sum() / tile.size >= min_size_fraction:
            bboxes_above_thresh.append(bbox.ravel())

    return bboxes_above_thresh

To test our function, with Imspector running in the background, let's apply it to the currently open image:

In [ ]:
im = specpy.get_application()

channel = 0
img = im.active_measurement().stack(channel).data().squeeze()

divide_into_tiles(img, threshold=4, num_divisions=2)

## Integration into a **autoSTED** pipeline

To integrate this function into an automated imaging pipeline, all we have to do is wrap it in a ```ROIDetectorWrapper``` in a callback that enqueuse higher-resolution sub-images.

**Note:** we only image one region (```start_callback``` is only called once as the initial callback to ```pipeline.run()```). By adding e.g. a ```SpiralOffsetGenerator``` and calling it after each overview, this could be easily extended to larger-scale imaging.

In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks import (
    JSONSettingsLoader,
    FOVSettingsGenerator,
    LocationRemover,
)
from autosted.detection import ROIDetectorWrapper
from autosted.utils.dict_utils import get_parameter_value_array_from_dict
from autosted.utils.parameter_constants import PIXEL_SIZE_PARAMETERS

# get current measurement from Imspector
im = specpy.get_application()
params = im.value_at("", specpy.ValueTree.Measurement).get()

# get pixel size of active measurement
pixel_size = get_parameter_value_array_from_dict(params, PIXEL_SIZE_PARAMETERS)

pipeline = AcquisitionPipeline("acquisition_data/pyramid", ["level0", "level1"])

# initial image at double pixel size (i.e. half resolution)
start_callback = AcquisitionTaskGenerator(
    "level0",
    LocationRemover(JSONSettingsLoader(params)),
    FOVSettingsGenerator(pixel_sizes=pixel_size * 2),
)

# get ROIs to image at level1 with original pixel size using divide_into_tiles
tile_callback = AcquisitionTaskGenerator(
    "level1",
    LocationRemover(JSONSettingsLoader(params)),
    ROIDetectorWrapper(divide_into_tiles, detection_kwargs={"threshold": 4}),
)

pipeline.add_callback(tile_callback, "level0")
pipeline.run(start_callback)

## Extending to multiple levels

Above, we run a two-level pipeline.

There is no reason to stop there - by simply adding another hierarchy level in the pipeline and a callback to sub-divide the images of the second level, we can add a third level (or more).

Note, that by default acquisition tasks of a later level take priority, thus our image pyramid would be scanned in a depth-first manner. This can be changed by assigning custom priorities for the levels.

In [ ]:
im = specpy.get_application()
params = im.value_at("", specpy.ValueTree.Measurement).get()

pixel_size = get_parameter_value_array_from_dict(params, PIXEL_SIZE_PARAMETERS)

pipeline = AcquisitionPipeline(
    "acquisition_data/pyramid", ["level0", "level1", "level2"]
)

# by default, increasing levels have a lower priority number, i.e. they will be imaged first
# by giving them increasing priorities, we can instead do a breadth-first traversal
pipeline.level_priorities = {"level0": 0, "level1": 1, "level2": 2}

# first overview at 9-fold subsampling
start_callback = AcquisitionTaskGenerator(
    "level0",
    LocationRemover(JSONSettingsLoader(params)),
    FOVSettingsGenerator(pixel_sizes=pixel_size * 9),
)

# second level at 3-fold subsampling
tile_callback_l1 = AcquisitionTaskGenerator(
    "level1",
    LocationRemover(JSONSettingsLoader(params)),
    FOVSettingsGenerator(pixel_sizes=pixel_size * 3),
    ROIDetectorWrapper(
        divide_into_tiles, detection_kwargs={"threshold": 5, "num_divisions": 3}
    ),
)

# 3rd level at original pixel size
tile_callback_l2 = AcquisitionTaskGenerator(
    "level2",
    LocationRemover(JSONSettingsLoader(params)),
    ROIDetectorWrapper(
        divide_into_tiles, detection_kwargs={"threshold": 5, "num_divisions": 3}
    ),
)

pipeline.add_callback(tile_callback_l1, "level0")
pipeline.add_callback(tile_callback_l2, "level1")
pipeline.run(start_callback)

## Visualizing acquired data

Here, we use image fusion functionality from ```calmutils``` to stitch the images of a given pyramid level into one large image.

We make use of the helper function ```approximate_pixel_shift_from_settings``` to get a virtual position of the images, i.e. combining scan and stage offsets (approximate due to rounding to nearest integer pixel).

In [ ]:
from autosted.utils.coordinate_utils import approximate_pixel_shift_from_settings
from calmutils.stitching.transform_helpers import translation_matrix
from calmutils.stitching.fusion import fuse_image

level = "level2"
channel = 0
configuration = 0

# get all measurement settings and images at selected level, config and channel
settings = [
    measurement.measurement_settings[configuration]
    for idx, measurement in pipeline.data.items()
    if idx[-1][0] == level
]
images = [
    measurement.data[configuration][channel].squeeze()
    for idx, measurement in pipeline.data.items()
    if idx[-1][0] == level
]

# get pixel shifts of all images relative to first
pixel_shifts = [
    approximate_pixel_shift_from_settings(settings[0], setting_i)
    for setting_i in settings
]

# to transformation matrix
is2d = images[0].ndim == 2
transforms = [translation_matrix(shift[(1 if is2d else 0) :]) for shift in pixel_shifts]

# fuse (low out-of-bounds value to better visualize non-imaged areas)
fused = fuse_image(images, transforms, oob_val=-5)

The result can be displayed using matplotlib or napari:

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(fused, cmap="magma", clim=(-5, 50))